---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [69]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

In [70]:
def print_full(df):
    pd.set_option('display.max_rows', len(df))
    display(df)
    pd.reset_option('display.max_rows')

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [71]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [72]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    uni_towns = pd.read_fwf('data/university_towns.txt', header=None, names=['State', 'RegionName'])
    uni_towns['RegionName'] = uni_towns['State']
    #solo dejar los nombres de estado en la columna 'State'
    uni_towns['State'].where(uni_towns['RegionName'].str.contains('[edit]', regex=False), inplace=True)
    #itertuples() is faster, but irrelevant here
    current_state = ''
    list_drop = []
    for index, row in uni_towns.iterrows():
        if type(row['State']) == str: 
            current_state = row['State']
            #armar lista de filas a remover luego
            list_drop.append(index)
            #Sacar edit de State
            current_state = current_state.replace('[edit]','')
        else:
            #Copiar el estado hasta en todas las filas que corresponde
            uni_towns['State'].iloc[index] = current_state
            #Remover
            uni_towns['RegionName'].iloc[index] = re.sub('\s\(.+', '', uni_towns['RegionName'].iloc[index])
    
    #uni_towns['State'] = uni_towns['State'].map(states)
    uni_towns.drop(index=list_drop, axis=0, inplace=True)
    return uni_towns

In [73]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('data/gdplev.xls', header=5,skiprows=214, names=['Quarter','GDP'],
                    usecols=[4,6] )
    #adding columns with diff and derivative (using diff only right now)
    gdp['GDP_diff'], gdp['GDP_deriv'] = gdp['GDP'].diff(), np.gradient(gdp['GDP'])
    #gdp.plot()
    #rows whose difference is negative
    gdp_neg = gdp[gdp['GDP_diff'] < 0]
    #find indexes of consecutive negative gdp
    gdp_neg_idx = [gdp_neg.index[i] for i in range(len(gdp_neg.index)-1) if gdp_neg.index[i+1]-gdp_neg.index[i]==1]
    
    return gdp['Quarter'][gdp_neg_idx[0]]


In [86]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('data/gdplev.xls', header=5,skiprows=214, names=['Quarter','GDP'],
                    usecols=[4,6] )
    #adding columns with diff and derivative (using diff only right now)
    gdp['GDP_diff'], gdp['data/GDP_deriv'] = gdp['GDP'].diff(), np.gradient(gdp['GDP'])

    rec_start_quarter = get_recession_start()
    rec_start_idx = gdp[gdp['Quarter'] == rec_start_quarter].index[0]
    #GDP with diff > 0, starting from recession quarter
    gdp_pos = gdp[gdp['GDP_diff'] > 0].loc[rec_start_idx:]
    
    
    #By definition, a recession ending is two consecutive quarters of GDP growth. 
    #So let's take the 2nd row     
    return gdp_pos.iloc[1][0]


In [85]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #read xls
    gdp = pd.read_excel('data/gdplev.xls', header=5,skiprows=214, names=['Quarter','GDP'],
                    usecols=[4,6] )
    #adding columns with diff and derivative (using diff only right now)
    #gdp['GDP_diff'], gdp['GDP_deriv'] = gdp['GDP'].diff(), np.gradient(gdp['GDP'])
    rec_start_quarter = get_recession_start()
    rec_start_idx = gdp[gdp['Quarter'] == rec_start_quarter].index[0]
    rec_end_quarter = get_recession_end()
    rec_end_idx = gdp[gdp['Quarter'] == rec_end_quarter].index[0]
    bottom_idx = gdp.iloc[rec_start_idx:rec_end_idx+1]['GDP'].idxmin()
    
    return gdp['Quarter'].iloc[bottom_idx]

In [76]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    houses = pd.read_csv('data/City_Zhvi_AllHomes.csv',encoding = "ISO-8859-1")
    houses['State'] = houses['State'].map(states)
    houses.set_index(["State","RegionName"], inplace=True)
    houses.sort_index(inplace=True)
    
    houses_sel = houses.loc[:,'2000-01':'2016-08']
    #convertir a datetime porque resample lo requiere
    houses_sel.columns = pd.to_datetime(houses_sel.columns, infer_datetime_format=True)

    #resample a quarters y mean
    houses_sel = houses_sel.resample('Q',axis=1).mean()
    #rename de nombre de columnas
    houses_sel.columns = pd.PeriodIndex(pd.to_datetime(houses_sel.columns), freq='Q')
    houses_sel.columns = houses_sel.columns.to_series().astype(str)
    houses_sel.rename(columns=lambda x: x.lower(), inplace = True)
    
    return houses_sel

In [92]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    bstart = hdata.columns[hdata.columns.get_loc(start) -1]
    
    hdata['ratio'] = hdata[bottom] - hdata[bstart]
    hdata = hdata[[bottom,bstart,'ratio']]
    hdata = hdata.reset_index()
    unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    hdata2 = pd.merge(hdata,unitowns_hdata,how='outer',on=['State','RegionName',bottom,bstart,'ratio'])
    hdata2['uni'] = hdata2['uni'].fillna(False)

    ut = hdata2[hdata2['uni'] == True]
    nut = hdata2[hdata2['uni'] == False]

    t,p = ttest_ind(ut['ratio'].dropna(),nut['ratio'].dropna())
    
    different = True if p < 0.01 else False

    better = "non-university town" if ut['ratio'].mean() < nut['ratio'].mean() else "university town"

    return different, p, better
run_ttest()

(True, 7.417614191647064e-05, 'university town')